# Tablesaw 

[Tablesaw](https://tablesaw.tech/) provides the ability to easily transform, summarize, and filter data, as well as computing descriptive statistics. It can also be used easily with libraries like Smile, which provides fundamental machine learning algorithms.

This notebook has some basic demos of how to use Tablesaw, including visualizing the results for which it uses the BeakerX interactive visualization APIs. Tablesaw also provides its own visualization APIs if you wish to do visualization outside of BeakerX. The notebook covers basic table manipulation, k-means clustering, linear regression, and fetching financial data.

In [ ]:
%%classpath add mvn
tech.tablesaw tablesaw-beakerx 0.35.0
com.jimmoores quandl-tablesaw 2.0.0
com.github.haifengl smile-core 1.5.2

In [ ]:
%import static tech.tablesaw.aggregate.AggregateFunctions.*
%import tech.tablesaw.api.*
%import tech.tablesaw.columns.*
%import smile.clustering.*
%import smile.regression.*

// display Tablesaw tables with BeakerX table display widget
tech.tablesaw.beakerx.TablesawDisplayer.register()

In [ ]:
tornadoes = Table.read().csv("../resources/data/tornadoes_2014.csv")

In [ ]:
//print dataset structure
tornadoes.structure()

In [ ]:
//get header names
tornadoes.columnNames()

In [ ]:
//displays the row and column counts
tornadoes.shape()

In [ ]:
//displays the first n rows
tornadoes.first(10)

In [ ]:
tornadoes.structure()

In [ ]:
//summarize the data in each column
tornadoes.summary()

In [ ]:
//Mapping operations
def month = tornadoes.dateColumn("Date").month()
tornadoes.addColumns(month);
tornadoes.columnNames()

In [ ]:
//Sorting by column
tornadoes.sortOn("-Fatalities")

In [ ]:
//Descriptive statistics
tornadoes.column("Fatalities").summary()

In [ ]:
//Performing totals and sub-totals
def injuriesByScale = tornadoes.summarize("Injuries", median).by("Scale")
injuriesByScale.setName("Median injuries by Tornado Scale")
injuriesByScale

In [ ]:
//Cross Tabs
tornadoes.xTabCounts("State", "Scale")

## K-means clustering

K-means is the most common form of “centroid” clustering. Unlike classification, clustering is an unsupervised learning method. The categories are not predetermined. Instead, the goal is to search for natural groupings in the dataset, such that the members of each group are similar to each other and different from the members of the other groups. The K represents the number of groups to find.

We’ll use a well known Scotch Whiskey dataset, which is used to cluster whiskeys according to their taste based on data collected from tasting notes. As always, we start by loading data and printing its structure.

In [ ]:
whiskeyData = Table.read().csv("../resources/data/whiskey.csv")
whiskeyData.structure()

In [ ]:
kMeans = new KMeans(whiskeyData.as().doubleMatrix("Body", "Sweetness", "Smoky", "Medicinal", "Tobacco", "Honey", "Spicy", "Winey", "Nutty", "Malty", "Fruity", "Floral"), 5)

In [ ]:
Table whiskeyClusters = Table.create("Clusters", whiskeyData.stringColumn("Distillery"), DoubleColumn.create("Cluster", kMeans.getClusterLabel()));
whiskeyClusters = whiskeyClusters.sortAscendingOn("Cluster", "Distillery");

## Play (Money)ball with Linear Regression

In baseball, you make the playoffs by winning more games than your rivals. The number of games the rivals win is out of your control so the A’s looked instead at how many wins it took historically to make the playoffs. They decided that 95 wins would give them a strong chance.  Here’s how we might check that assumption in Tablesaw.

In [ ]:
baseball = Table.read().csv("../resources/data/baseball.csv");

// filter to the data available at the start of the 2002 season
moneyball = baseball.where(baseball.numberColumn("year").isLessThan(2002));
wins = moneyball.nCol("W");
year = moneyball.nCol("Year");
playoffs = moneyball.column("Playoffs");
runDifference = moneyball.numberColumn("RS").subtract(moneyball.numberColumn("RA")).setName("RD");
moneyball.addColumns(runDifference);

def Plot = new Plot(title: "RD x Wins", xLabel:"RD", yLabel: "W")
Plot << new Points(x: moneyball.numberColumn("RD").asDoubleArray(), y: moneyball.numberColumn("W").asDoubleArray())

In [ ]:
winsModel = new OLS(moneyball.select("W", "RD").smile().numericDataset("RD"));

In [ ]:
runsScored = new OLS(moneyball.select("OBP", "SLG", "RS").smile().numericDataset("RS"));

In [ ]:
new Histogram(xLabel:"X",
              yLabel:"Proportion",
              data: Arrays.asList(runsScored.residuals()),
              binCount: 25);

## Financial and Economic Data

You can fetch data from [Quandl](https://www.quandl.com/) and load it directly into Tablesaw

In [ ]:
%import com.jimmoores.quandl.DataSetRequest
%import com.jimmoores.quandl.tablesaw.*

In [ ]:
TableSawQuandlSession session = TableSawQuandlSession.create();
Table table = session.getDataSet(DataSetRequest.Builder.of("WIKI/AAPL").build());
// Create a new column containing the year
NumberColumn yearColumn = table.dateColumn("Date").year();
yearColumn.setName("Year");
table.addColumns(yearColumn);
// Create max, min and total volume tables aggregated by year
Table summaryMax = table.summarize("Adj. Close", max).by("year");
Table summaryMin = table.summarize("Adj. Close", min).by("year");
Table summaryVolume = table.summarize("Volume", sum).by("year");
// Create a new table from each of these
summary = Table.create("Summary", summaryMax.column(0), summaryMax.column(1), 
                       summaryMin.column(1), summaryVolume.column(1));
// Add back a DateColumn to the summary...will be used for plotting
DateColumn yearDates = DateColumn.create("YearDate");
for (year in summary.column('Year')) {
    yearDates.append(java.time.LocalDate.of((int) year, 1, 1));
}
summary.addColumns(yearDates)

summary

In [ ]:
years = summary.column('YearDate').collect()

plot = new TimePlot(title: 'Price Chart for AAPL', xLabel: 'Time', yLabel: 'Max [Adj. Close]')
plot << new YAxis(label: 'Volume')
plot << new Points(x: years, y: summary.column('Max [Adj. Close]').collect())
plot << new Line(x: years, y: summary.column('Max [Adj. Close]').collect(), color: Color.blue)
plot << new Stems(x: years, y: summary.column('Sum [Volume]').collect(), yAxis: 'Volume')